In [14]:
import pandas as pd
import sqlite3
import math 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from IPython.core.display import display, HTML


In [15]:
conn = sqlite3.connect('database.db')

In [16]:
c = conn.cursor()

## Desain Database
<img src="dw.png">

In [17]:
c.executescript('drop table if exists kontraktor')
c.executescript('drop table if exists produk')
c.executescript('drop table if exists time')
c.executescript('drop table if exists sales')
c.execute("CREATE TABLE IF NOT EXISTS 'kontraktor' ('kode_kontraktor' INT, 'nama_kontraktor' VARCHAR(255) NULL,'kontak_person' VARCHAR(255) NULL, PRIMARY KEY ('kode_kontraktor'))")
c.execute("CREATE TABLE IF NOT EXISTS 'produk' ('id_produk' INT, 'nama_produk' VARCHAR(255) NULL,'kategori_produk' VARCHAR(255) NULL, PRIMARY KEY ('id_produk'))")
c.execute("CREATE TABLE IF NOT EXISTS 'time' ('id_time' INT, 'bulan' VARCHAR(255) NULL, PRIMARY KEY ('id_time'))")
c.execute("CREATE TABLE IF NOT EXISTS 'sales' ( 'id_fact' INT, 'id_time' INT, 'kode_kontraktor' INT, 'id_produk' INT, 'jumlah_pesanan' INT, PRIMARY KEY ('id_fact'), FOREIGN KEY ('id_time') REFERENCES 'time' ('id_time') ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ('kode_kontraktor') REFERENCES 'kontraktor' ('kode_kontraktor') ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ('id_produk') REFERENCES 'produk' ('id_produk') ON DELETE NO ACTION ON UPDATE NO ACTION)")

In [18]:
df = pd.read_csv("data dw (1).csv")
df

,Bulan Pemesanan,Nama Produk,Kontraktor,Kategori Produk,Quantity,Contact person
0,Januari,Lampu LED 20 Watt,CV. Selaras,Uniled 20 Watt,10 Unit,Bapak Fauzi
1,Januari,Lampu LED 30 Watt,CV. Selaras,Uniled 30 Watt,30 Unit,Bapak Fauzi
2,Januari,Solar Cell 50 WP,CV. Selaras,SC 50 WP,500 unit,Bapak Fauzi
3,Januari,Regulator,CV. Selaras,BCR,70 Unit,Bapak Fauzi
4,Januari,Lampu LED 20 Watt,PT. Panji Makmur,Uniled 20 Watt,40 Unit,Bapak Rosidi
...,...,...,...,...,...,...
145,Desember,Solar Cell 50 WP,PT. Aditarwan,SC 50 WP,60 Unit,Bapak Aditya
146,Desember,Penerangan Jalan Umum 60 Watt,CV. Metec,PJU 60 Watt,50 Unit,Bapak Rudi
147,Desember,Lampu Land Scape 40 Watt,CV. Metec,LLS 40 Watt,30 Unit,Bapak Rudi
148,Desember,Lampu Heavy Duty 40 Watt,CV. Metec,LHD 40 Watt,20 Unit,Bapak Rudi


In [19]:
time = []
produk = []
kontraktor = []
sales = []
for i,j in df.iterrows():
    if j[0] not in time:
        time.append(j[0])
        
    t = -1
    for index, value in enumerate(time):
        if value == j[0]:
            t = index+1
            break
        
    if [j[1],j[3], t] not in produk:
        produk.append([j[1], j[3], t])
    
    if [j[2],j[5], t] not in kontraktor:
        kontraktor.append([j[2],j[5], t])


sales = []
for i,j in df.iterrows():   
    t = -1
    for index, value in enumerate(time):
        if value[0] == j[0]:
            t = index+1
            break
    
    k = -1
    for index, value in enumerate(kontraktor):
        if value[0] == j[2]:
            s = index+1
            break
        
    prod = -1
    for index, value in enumerate(produk):
        if value[0] == j[3]:
            prod = index+1
    
    sales.append([t, k, prod, j[4]])

In [28]:
c.execute("DELETE FROM KONTRAKTOR")
for index, i in enumerate(kontraktor): 
    c.execute("INSERT INTO KONTRAKTOR VALUES(" + str(index+1) + ", '" + str(i[0]) + "', '" + str(i[1]) + "')")

c.execute("DELETE FROM PRODUK")
for index, i in enumerate(produk):
    c.execute("INSERT INTO PRODUK VALUES(" + str(index+1) + ", '" + str(i[0]) + "','" + str(i[1]) + "')")
    
c.execute("DELETE FROM TIME")
for index, i in enumerate(time): 
    c.execute("INSERT INTO TIME VALUES(" + str(index+1) + ", '" + str(i[0]) + "')")
    

c.execute("DELETE FROM SALES")
for index, i in enumerate(sales):
    id_time = i[0]
    kode_kontraktor = i[1]
    id_produk = i[2]
    c.execute("INSERT INTO SALES VALUES(" + str(index+1) + "," + str(id_time) + "," + str(kode_kontraktor) + "," + str(id_produk) + ", '" + str(i[3]) + "')")
    
conn.commit()

In [33]:
c.execute('''SELECT * FROM SALES''')

SALES = []

for row in c.fetchall():
    SALES.append(row)
SALES


[(1, -1, -1, -1, '10 Unit'),
 (2, -1, -1, -1, '30 Unit'),
 (3, -1, -1, -1, '500 unit'),
 (4, -1, -1, -1, '70 Unit'),
 (5, -1, -1, -1, '40 Unit'),
 (6, -1, -1, -1, '65 Unit'),
 (7, -1, -1, -1, '20 Unit'),
 (8, -1, -1, -1, '30 Unit'),
 (9, -1, -1, -1, '150 Unit'),
 (10, -1, -1, -1, '30 Unit'),
 (11, -1, -1, -1, '150 Unit'),
 (12, -1, -1, -1, '70 Unit'),
 (13, -1, -1, -1, '40 Unit'),
 (14, -1, -1, -1, '30 Unit'),
 (15, -1, -1, -1, '60 Unit'),
 (16, -1, -1, -1, '10 Unit'),
 (17, -1, -1, -1, '20 Unit'),
 (18, -1, -1, -1, '50 Unit'),
 (19, -1, -1, -1, '300 Unit'),
 (20, -1, -1, -1, '150 Unit'),
 (21, -1, -1, -1, '200 Unit'),
 (22, -1, -1, -1, '600 Unit'),
 (23, -1, -1, -1, '450 Unit'),
 (24, -1, -1, -1, '250 Unit'),
 (25, -1, -1, -1, '50 Unit'),
 (26, -1, -1, -1, '40 Unit'),
 (27, -1, -1, -1, '150 Unit'),
 (28, -1, -1, -1, '150 Unit'),
 (29, -1, -1, -1, '150 Unit'),
 (30, -1, -1, -1, '150 Unit'),
 (31, -1, -1, -1, '100 Unit'),
 (32, -1, -1, -1, '70 Unit'),
 (33, -1, -1, -1, '250 Unit'),
 (34